In [1]:
from pybtex.database import parse_file
from pybtex.plugin import find_plugin
from pprint import pprint

bibfile = "../_data/publications/core_publications.bib"

In [2]:
bib_data = parse_file(bibfile)
alpha = find_plugin("pybtex.style.formatting", "alpha")
HTML = find_plugin('pybtex.backends', 'html')()
formatted_bib = alpha().format_bibliography(bib_data=bib_data)
html_entries = [entry.text.render(HTML) for entry in formatted_bib]
html_str = "<ol>\n"
for html_entry in html_entries:
    html_str += f"\t<li>{html_entry}</li>\n"
html_str += "</ol>"

In [3]:
with open("../_includes/publications/core_publications.html", "w") as fo:
    fo.write(html_str)